In [ ]:
#라이브러리 import 해주기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sb
import io


In [ ]:
!sudo apt-get install -y fonts-nanum

!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

plt.rc('font', family='NanumBarunGothic')

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 15 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 2s (5,509 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 146442 files and d

In [ ]:
#구글 드라이브를 사용하여 csv 데이터 불러오기
from google.colab import drive
drive.mount('/content/drive')
filename = '/content/drive/MyDrive/data.csv'
data = pd.read_csv(filename , encoding = 'cp949')
data

Mounted at /content/drive


,작업번호,마킹 길이 (m),절단 길이 (m),철판두께,철판재질,절단갯수,작업장,절단장비,마킹시간(초),절단시간(초)
0,1,46.0,21.9,30.0,DZB,1,23,PL233,194,827
1,2,33.9,111.8,12.0,A,46,23,PL233,324,2906
2,3,49.5,33.5,13.0,A,2,23,PL234,241,1003
3,4,75.7,174.9,13.0,A,35,23,PL233,483,4412
4,5,66.5,66.6,13.0,A,7,23,PL233,331,1601
...,...,...,...,...,...,...,...,...,...,...
162320,162337,139.8,162.0,19.0,AH32,31,23,PL231,827,2899
162321,162338,18.5,28.9,16.0,A,56,25,PL252,165,930
162322,162339,63.7,25.6,15.0,AH32,1,26,PL261,345,696
162323,162340,94.7,61.4,19.0,AH32,2,26,PL261,0,2155


In [ ]:
def outliers_iqr(data):
  q1, q3 = np.percentile(data,[25,75])
  iqr = q3 - q1
  lower_bound = q1 - (iqr * 1.5)
  upper_bound = q3 + (iqr * 1.5)

  return np.where((data > upper_bound)|(data < lower_bound))

markingtime_outlier_index = outliers_iqr(data['마킹시간(초)'])[0]
cuttingtime_outlier_index = outliers_iqr(data['절단시간(초)'])[0]

print(data.loc[markingtime_outlier_index, '마킹시간(초)'])
print(data.loc[cuttingtime_outlier_index, '절단시간(초)'])

33        2409
35        1523
36        1053
47        1162
63        1708
          ... 
162193    1766
162206    1034
162251    1234
162284    1213
162303    1114
Name: 마킹시간(초), Length: 6292, dtype: int64
7          5667
9          9549
10         6502
11         6463
63         5381
          ...  
162288     8644
162290    10816
162296     8261
162305     6491
162314     7130
Name: 절단시간(초), Length: 8542, dtype: int64


In [ ]:
data_outlier_index = np.concatenate((markingtime_outlier_index,cuttingtime_outlier_index), axis=None)
print(len(data_outlier_index))

14834


In [ ]:
data_not_outlier_index = []
for i in data.index:

  if i not in data_outlier_index :
    data_not_outlier_index.append(i)

In [ ]:
data_clean = data.loc[data_not_outlier_index]
data_clean = data_clean.reset_index(drop=True)
data_clean

,작업번호,마킹 길이 (m),절단 길이 (m),철판두께,철판재질,절단갯수,작업장,절단장비,마킹시간(초),절단시간(초)
0,1,46.0,21.9,30.0,DZB,1,23,PL233,194,827
1,2,33.9,111.8,12.0,A,46,23,PL233,324,2906
2,3,49.5,33.5,13.0,A,2,23,PL234,241,1003
3,4,75.7,174.9,13.0,A,35,23,PL233,483,4412
4,5,66.5,66.6,13.0,A,7,23,PL233,331,1601
...,...,...,...,...,...,...,...,...,...,...
149264,162337,139.8,162.0,19.0,AH32,31,23,PL231,827,2899
149265,162338,18.5,28.9,16.0,A,56,25,PL252,165,930
149266,162339,63.7,25.6,15.0,AH32,1,26,PL261,345,696
149267,162340,94.7,61.4,19.0,AH32,2,26,PL261,0,2155


In [ ]:
#전처리를 마친 데이터
data_0 = data_clean[(data_clean['절단시간(초)'] == 0) | (data_clean['마킹시간(초)'] == 0)].index
data_completed = data_clean.drop(data_0)
data_completed

,작업번호,마킹 길이 (m),절단 길이 (m),철판두께,철판재질,절단갯수,작업장,절단장비,마킹시간(초),절단시간(초)
0,1,46.0,21.9,30.0,DZB,1,23,PL233,194,827
1,2,33.9,111.8,12.0,A,46,23,PL233,324,2906
2,3,49.5,33.5,13.0,A,2,23,PL234,241,1003
3,4,75.7,174.9,13.0,A,35,23,PL233,483,4412
4,5,66.5,66.6,13.0,A,7,23,PL233,331,1601
...,...,...,...,...,...,...,...,...,...,...
149263,162336,75.2,124.0,16.0,AH32,33,23,PL234,464,3077
149264,162337,139.8,162.0,19.0,AH32,31,23,PL231,827,2899
149265,162338,18.5,28.9,16.0,A,56,25,PL252,165,930
149266,162339,63.7,25.6,15.0,AH32,1,26,PL261,345,696


In [ ]:
#랜덤포레스트를 통한 마킹시간과 절단시간 예측.
#우선 독립변수와 종속변수 사이의 상관관계를 분석해보도록 한다
corr_data_completed = data_completed.corr(method = 'pearson')
corr_data_completed
#상관관계를 살펴보니 마킹시간 -> 마킹길이, 절단길이와 가장 큰 상관관계를 보인다./ 절단시간 -> 절단 길이, 절단갯수와 가장 큰 상관관계를 보인다.
#다만 지금은 철판재질과 절단장비를 고려하지 않은 것이다.
corr_completed = corr_data_completed.drop(['작업장','작업번호'], axis=0)
corr_completed2 = corr_completed.drop(['작업장','작업번호'], axis=1)
corr_completed2

,마킹 길이 (m),절단 길이 (m),철판두께,절단갯수,마킹시간(초),절단시간(초)
마킹 길이 (m),1.000000,0.064779,0.040899,-0.217742,0.708683,0.043909
절단 길이 (m),0.064779,1.000000,-0.283597,0.532988,0.336746,0.893000
철판두께,0.040899,-0.283597,1.000000,-0.310356,-0.094868,-0.024261
절단갯수,-0.217742,0.532988,-0.310356,1.000000,-0.006328,0.485044
마킹시간(초),0.708683,0.336746,-0.094868,-0.006328,1.000000,0.336412
절단시간(초),0.043909,0.893000,-0.024261,0.485044,0.336412,1.000000


In [ ]:
#작업장과 절단시간을 배제하고 범주형 변수로는 철판재질만을 사용한 모델입니다.
df_cutting_time = data_completed[['절단 길이 (m)','절단갯수','철판재질','절단시간(초)']]
df_marking_time = data_completed[['마킹 길이 (m)','절단 길이 (m)','철판재질','마킹시간(초)' ]]
print(df_cutting_time)
print(df_marking_time)

        절단 길이 (m)  절단갯수      철판재질  절단시간(초)
0            21.9     1   DZB          827
1           111.8    46  A            2906
2            33.5     2  A            1003
3           174.9    35  A            4412
4            66.6     7  A            1601
...           ...   ...       ...      ...
149263      124.0    33  AH32         3077
149264      162.0    31  AH32         2899
149265       28.9    56  A             930
149266       25.6     1  AH32          696
149268       20.7     8  DH32          678

[144407 rows x 4 columns]
        마킹 길이 (m)  절단 길이 (m)      철판재질  마킹시간(초)
0            46.0       21.9   DZB          194
1            33.9      111.8  A             324
2            49.5       33.5  A             241
3            75.7      174.9  A             483
4            66.5       66.6  A             331
...           ...        ...       ...      ...
149263       75.2      124.0  AH32          464
149264      139.8      162.0  AH32          827
149265       18.5       2

In [ ]:
#랜덤포레스트를 돌리기 위해선 범주형 변수를 더미화 해주어야 한다.
data_cutting_dummy_mat = pd.get_dummies(df_cutting_time, columns = ['철판재질'])
data_cutting_dummy_mat

,절단 길이 (m),절단갯수,절단시간(초),철판재질_32M60C,철판재질_9NI,철판재질_A,철판재질_AA,철판재질_AAH32,철판재질_AAH32-TM,철판재질_AAH36,철판재질_AE,철판재질_AEH32-TM,철판재질_AH32,철판재질_AH32-TM,철판재질_AH32TL,철판재질_AH36,철판재질_AH36-TM,철판재질_AH36TL,철판재질_AH40-TM,철판재질_B,철판재질_BA,철판재질_BAH32,철판재질_BAH36,철판재질_BZ,철판재질_BZB,철판재질_D,철판재질_DH32,철판재질_DH32-TM,철판재질_DH32TL,철판재질_DH36,철판재질_DH36-TM,철판재질_DH36TL,철판재질_DH40-TM,철판재질_DZB,철판재질_DZB,철판재질_DZB-TM,철판재질_E,철판재질_E-TM,철판재질_EH32,철판재질_EH32-TM,철판재질_EH32TL,철판재질_EH36,철판재질_EH36-TM,철판재질_EH40-TM,철판재질_ETL,철판재질_EZ,철판재질_EZB,철판재질_EZB-TM,철판재질_GA,철판재질_KA,철판재질_KAH32,철판재질_KAH32-TM,철판재질_LA,철판재질_LAH32,철판재질_LAH32-TM,철판재질_LAH36,철판재질_LDH32,철판재질_LDH32-TM,철판재질_LEH32-TM,철판재질_NA,철판재질_NAH32,철판재질_NAH32-TM,철판재질_NB,철판재질_NEH32,철판재질_NV32M60C,철판재질_RA,철판재질_RAH32,철판재질_RAH32-TM,철판재질_RAH36,철판재질_SS400,철판재질_UA,철판재질_UAH32,철판재질_UDH32,철판재질_VA,철판재질_VAH32,철판재질_VAH32-TM,철판재질_VAH36,철판재질_VB,철판재질_VDH32-TM,철판재질_VH32O55C
0,21.9,1,827,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,111.8,46,2906,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,33.5,2,1003,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,174.9,35,4412,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,66.6,7,1601,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149263,124.0,33,3077,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
149264,162.0,31,2899,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
149265,28.9,56,930,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
149266,25.6,1,696,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#랜덤포레스트 데이터셋 분리하기
from sklearn.model_selection import train_test_split

X1=data_cutting_dummy_mat.drop(['절단시간(초)'], axis=1)
Y1=data_cutting_dummy_mat['절단시간(초)']


X1_train,X1_test,Y1_train,Y1_test = train_test_split(X1,Y1,test_size=0.4,shuffle=True,random_state=1004)
print(len(X1_train))
print(len(Y1_train))
print(len(X1_test))
print(len(Y1_test))

86644
86644
57763
57763


In [ ]:
#절단시간 예측

from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.metrics import explained_variance_score

forest = RandomForestRegressor()
forest.fit(X1_train, Y1_train)
Y1_pred = forest.predict(X1_test)

print(forest.score(X1_train, Y1_train))
print(explained_variance_score(Y1_pred, Y1_test))

0.9278239803179082
0.8164357698831436


In [ ]:
data_marking_dummy_mat = pd.get_dummies(df_marking_time, columns = ['철판재질'])

df2 = data_marking_dummy_mat.copy()


X2=df2.drop(['마킹시간(초)'], axis=1)
Y2=df2['마킹시간(초)']


X2_train,X2_test,Y2_train,Y2_test = train_test_split(X2,Y2,test_size=0.4,shuffle=True,random_state=1004)
print(len(X2_train))
print(len(Y2_train))
print(len(X2_test))
print(len(Y2_test))
print(X2)


86644
86644
57763
57763
        마킹 길이 (m)  절단 길이 (m)  ...  철판재질_VDH32-TM  철판재질_VH32O55C
0            46.0       21.9  ...              0              0
1            33.9      111.8  ...              0              0
2            49.5       33.5  ...              0              0
3            75.7      174.9  ...              0              0
4            66.5       66.6  ...              0              0
...           ...        ...  ...            ...            ...
149263       75.2      124.0  ...              0              0
149264      139.8      162.0  ...              0              0
149265       18.5       28.9  ...              0              0
149266       63.7       25.6  ...              0              0
149268       11.4       20.7  ...              0              0

[144407 rows x 79 columns]


In [ ]:
forest = RandomForestRegressor()
forest.fit(X2_train, Y2_train)
Y2_pred = forest.predict(X2_test)

print(forest.score(X2_train, Y2_train))

0.890553753529724


In [ ]:
from sklearn.metrics import explained_variance_score
print(explained_variance_score(Y2_pred, Y2_test))

0.5102719403232002
